In [25]:
import boto3 #used to connect to aws services to work with it
import pandas as pd
from io import StringIO # to encode data in binary form
import time
import warnings
warnings.filterwarnings("ignore")

In [26]:
AWS_ACCESS_KEY ="XXXXXXXXXXXXX"
AWS_SECRET_KEY = "XXXXXXXXXXXXXXXX"
AWS_REGION = "us-east-1"
SCHEMA_NAME = "covid_dataset"
S3_STAGING_DIR = "s3://meenakshi-test-bucket/output/"
S3_BUCKET_NAME = "meenakshi-test-bucket"
S3_OUTPUT_DIRECTORY = "output"

In [27]:
#connect to athena and query data

In [28]:
athena_client = boto3.client(
                "athena",
                aws_access_key_id = AWS_ACCESS_KEY,
                aws_secret_access_key = AWS_SECRET_KEY,
                region_name = AWS_REGION,)

In [38]:
import boto3
import os
import pandas as pd

# Set up AWS credentials and region
# Configure AWS SDK
s3_client = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY, region_name=AWS_REGION)

#dictionary to store the after querying tables
table_names = ["enigma_jhud", "nytimes_data_in_usa_countyus_county","covid_19_testing_data_states_dailystates_daily","covid_19_testing_data_us_dailyus_daily","covid_19_testing_data_us_total_latestus_total_latest","nytimes_data_in_usaus_states","rearc_usa_hospital_beds","static_datasets_countrycodecountrycode","static_datasets_countypopulationcountypopulation","static_datasets_state_abvstate_abv"]

query_execution_responses = []

for table_name in table_names:
    response = athena_client.start_query_execution(
        QueryString=f"SELECT * FROM {table_name}",
        QueryExecutionContext={"Database": SCHEMA_NAME},
        ResultConfiguration={
            "OutputLocation": S3_STAGING_DIR,
            "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
        },
    )
    query_execution_responses.append(response)

all_files_downloaded = True  # Flag to check if all files are downloaded

for i in range(len(table_names)):
    table_name = table_names[i]
    query_execution_id = query_execution_responses[i]['QueryExecutionId']

    query_execution_result = athena_client.get_query_execution(
        QueryExecutionId=query_execution_id
    )
    
    s3_object_key = query_execution_result['QueryExecution']['ResultConfiguration']['OutputLocation']
    s3_object_key = s3_object_key.replace('s3://meenakshi-test-bucket/', '')
    
    file_name = table_name
    file_extension = '.csv'
    full_file_name = file_name + file_extension
    # Download the file from S3
    s3_client.download_file('meenakshi-test-bucket', s3_object_key, full_file_name)
    print(f"File downloaded: {full_file_name}")
    
    # Check if the file exists
    if not os.path.exists(full_file_name):
        all_files_downloaded = False
        break

if all_files_downloaded:
    print("All files downloaded successfully.")
    
    # Delete the files from S3
    for i in range(len(table_names)):
        table_name = table_names[i]
        query_execution_id = query_execution_responses[i]['QueryExecutionId']
        file_name = query_execution_id + ".csv"
        file_metadata_name = query_execution_id + ".csv.metadata"
        s3_object_key = f"output/{file_name}"
        s3_metadata_object_key = f"output/{file_metadata_name}"
        print(s3_object_key)
        print(s3_metadata_object_key)
        s3_client.delete_object(Bucket='meenakshi-test-bucket', Key=s3_object_key)
        s3_client.delete_object(Bucket='meenakshi-test-bucket', Key=s3_metadata_object_key)
        print(f"File deleted from S3: {s3_object_key}")
        print(f"File deleted from S3: {s3_metadata_object_key}")
else:
    print("Not all files were downloaded. Check the file download process.")



File downloaded: enigma_jhud.csv
File downloaded: nytimes_data_in_usa_countyus_county.csv
File downloaded: covid_19_testing_data_states_dailystates_daily.csv
File downloaded: covid_19_testing_data_us_dailyus_daily.csv
File downloaded: covid_19_testing_data_us_total_latestus_total_latest.csv
File downloaded: nytimes_data_in_usaus_states.csv
File downloaded: rearc_usa_hospital_beds.csv
File downloaded: static_datasets_countrycodecountrycode.csv
File downloaded: static_datasets_countypopulationcountypopulation.csv
File downloaded: static_datasets_state_abvstate_abv.csv
All files downloaded successfully.
output/5e01d09a-ed31-4337-84dc-0a84da36ea82.csv
output/5e01d09a-ed31-4337-84dc-0a84da36ea82.csv.metadata
File deleted from S3: output/5e01d09a-ed31-4337-84dc-0a84da36ea82.csv
File deleted from S3: output/5e01d09a-ed31-4337-84dc-0a84da36ea82.csv.metadata
output/adee491f-5c71-4a01-8f06-a8490060a6fe.csv
output/adee491f-5c71-4a01-8f06-a8490060a6fe.csv.metadata
File deleted from S3: output/adee

In [39]:
import pandas as pd

table_data = {}  # Dictionary to store table data

table_names = ["enigma_jhud", "nytimes_data_in_usa_countyus_county","covid_19_testing_data_states_dailystates_daily","covid_19_testing_data_us_dailyus_daily","covid_19_testing_data_us_total_latestus_total_latest","nytimes_data_in_usaus_states","rearc_usa_hospital_beds","static_datasets_countrycodecountrycode","static_datasets_countypopulationcountypopulation","static_datasets_state_abvstate_abv"]

for table_name in table_names:
    file_name = table_name + ".csv"
    
    # Read the CSV file into a DataFrame
    table_data[table_name] = pd.read_csv(file_name)
    print(f"CSV file read: {file_name} for table: {table_name}")

# Access the table data using table names as keys
for table_name, data in table_data.items():
    # Perform operations on the table data
    print(f"Table: {table_name}")
    print(data.head())  # Example: Print the first few rows of the table
    print()


CSV file read: enigma_jhud.csv for table: enigma_jhud
CSV file read: nytimes_data_in_usa_countyus_county.csv for table: nytimes_data_in_usa_countyus_county
CSV file read: covid_19_testing_data_states_dailystates_daily.csv for table: covid_19_testing_data_states_dailystates_daily
CSV file read: covid_19_testing_data_us_dailyus_daily.csv for table: covid_19_testing_data_us_dailyus_daily
CSV file read: covid_19_testing_data_us_total_latestus_total_latest.csv for table: covid_19_testing_data_us_total_latestus_total_latest
CSV file read: nytimes_data_in_usaus_states.csv for table: nytimes_data_in_usaus_states
CSV file read: rearc_usa_hospital_beds.csv for table: rearc_usa_hospital_beds
CSV file read: static_datasets_countrycodecountrycode.csv for table: static_datasets_countrycodecountrycode
CSV file read: static_datasets_countypopulationcountypopulation.csv for table: static_datasets_countypopulationcountypopulation
CSV file read: static_datasets_state_abvstate_abv.csv for table: static_da

In [48]:
# Read the downloaded CSV file into a DataFrame

In [40]:
enigma_jhud_data = table_data["enigma_jhud"]
nytimes_data_usa_county = table_data["nytimes_data_in_usa_countyus_county"]
covid_testing_states_daily = table_data["covid_19_testing_data_states_dailystates_daily"]
covid_19_testing_data_us = table_data["covid_19_testing_data_us_dailyus_daily"]
covid_19_testing_data_us_total_latest = table_data["covid_19_testing_data_us_total_latestus_total_latest"]
nytimes_data_in_usa_states = table_data["nytimes_data_in_usaus_states"]
rearc_usa_hospital_beds = table_data["rearc_usa_hospital_beds"]
static_datasets_countrycode = table_data["static_datasets_countrycodecountrycode"]
static_datasets_countypopulation = table_data["static_datasets_countypopulationcountypopulation"]
static_datasets_state_abv = table_data["static_datasets_state_abvstate_abv"]

In [41]:
static_datasets_state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [42]:
new_header = static_datasets_state_abv.iloc[0] 
new_header

col0           State
col1    Abbreviation
Name: 0, dtype: object

In [43]:
static_datasets_state_abv = static_datasets_state_abv[1:]
static_datasets_state_abv.columns = new_header
static_datasets_state_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [44]:
factcovid_1 = enigma_jhud_data[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active',]]
factcovid_2 = covid_testing_states_daily[['fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid = pd.merge(factcovid_1,factcovid_2, on = 'fips', how = 'inner')
factCovid.tail()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
6974,72.0,Puerto Rico,US,100.0,3.0,0.0,0.0,20210123,90073,305972.0,325.0,NaN,NaN
6975,72.0,Puerto Rico,US,100.0,3.0,0.0,0.0,20210122,89282,305972.0,341.0,NaN,NaN
6976,72.0,Puerto Rico,US,100.0,3.0,0.0,0.0,20210121,88728,305972.0,344.0,NaN,NaN
6977,72.0,Puerto Rico,US,100.0,3.0,0.0,0.0,20210120,88513,305972.0,331.0,NaN,NaN
6978,72.0,Puerto Rico,US,100.0,3.0,0.0,0.0,20210119,88373,305972.0,351.0,NaN,NaN


In [45]:
enigma_jhud_data.columns

Index(['fips', 'admin2', 'province_state', 'country_region', 'last_update',
       'latitude', 'longitude', 'confirmed', 'deaths', 'recovered', 'active',
       'combined_key', 'partition_0'],
      dtype='object')

In [46]:
nytimes_data_usa_county.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [47]:
dim_region_1 = enigma_jhud_data[['fips', 'province_state', 'country_region','latitude', 'longitude']]
dim_region_2 = nytimes_data_usa_county[['county', 'state', 'fips']]
dimRegion = pd.merge(dim_region_1, dim_region_2, on = 'fips', how = 'inner')
dimRegion.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York


In [48]:
rearc_usa_hospital_beds.columns

Index(['objectid', 'hospital_name', 'hospital_type', 'hq_address',
       'hq_address1', 'hq_city', 'hq_state', 'hq_zip_code', 'county_name',
       'state_name', 'state_fips', 'cnty_fips', 'fips', 'num_licensed_beds',
       'num_staffed_beds', 'num_icu_beds', 'adult_icu_beds', 'pedi_icu_beds',
       'bed_utilization', 'avg_ventilator_usage',
       'potential_increase_in_bed_capac', 'latitude', 'longtitude'],
      dtype='object')

In [49]:
dim_hospital = rearc_usa_hospital_beds[[ 'fips','state_name','latitude', 'longtitude','hq_address','hospital_name', 'hospital_type','hq_city', 'hq_state']]
dim_hospital.head()

,fips,state_name,latitude,longtitude,hq_address,hospital_name,hospital_type,hq_city,hq_state
0,4013,Arizona,33.495498,-112.066157,650 E Indian School Rd,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,Phoenix,AZ
1,4019,Arizona,32.181263,-110.965885,3601 S 6th Ave,Southern Arizona VA Health Care System,VA Hospital,Tucson,AZ
2,6019,California,36.773324,-119.779742,2615 E Clinton Ave,VA Central California Health Care System,VA Hospital,Fresno,CA
3,9009,Connecticut,41.284400,-72.957610,950 Campbell Ave,VA Connecticut Healthcare System - West Haven ...,VA Hospital,West Haven,CT
4,10003,Delaware,39.740206,-75.606532,1601 Kirkwood Hwy,Wilmington VA Medical Center,VA Hospital,Wilmington,DE


In [50]:
dim_date = covid_testing_states_daily[['fips','date']]
dim_date.head()

,fips,date
0,49.0,20210220
1,51.0,20210220
2,78.0,20210220
3,50.0,20210220
4,53.0,20210220


In [51]:
dim_date['date'] = pd.to_datetime(dim_date['date'],format = '%Y%m%d')
dim_date.head()

,fips,date
0,49.0,2021-02-20
1,51.0,2021-02-20
2,78.0,2021-02-20
3,50.0,2021-02-20
4,53.0,2021-02-20


In [52]:
dim_date['year'] = dim_date['date'].dt.year
dim_date['month'] = dim_date['date'].dt.month
dim_date['dayOfWeek'] = dim_date['date'].dt.dayofweek
dim_date.head()

,fips,date,year,month,dayOfWeek
0,49.0,2021-02-20,2021,2,5
1,51.0,2021-02-20,2021,2,5
2,78.0,2021-02-20,2021,2,5
3,50.0,2021-02-20,2021,2,5
4,53.0,2021-02-20,2021,2,5


In [80]:
def upload_dataframe_to_s3(dataframe, file_path):
    # Convert DataFrame to CSV
    csv_data = dataframe.to_csv(index=False)

    # Create a session using your AWS credentials
    session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)

    # Create an S3 client
    s3_client = session.client('s3')

    # Define the S3 bucket and file path
    bucket_name = 'meenakshi-test-bucket'
    key = file_path

    # Check if the file already exists in the bucket
    try:
        s3_client.head_object(Bucket=bucket_name, Key=key)
        print(f'File {key} already exists. Deleting it...')
        s3_client.delete_object(Bucket=bucket_name, Key=key)
    except:
        pass

    # Upload the data to S3
    s3_client.put_object(Body=csv_data, Bucket=bucket_name, Key=key)

# Define a list of file paths for different CSV files
file_paths = ['output/dim_date.csv', 'output/dim_hospital.csv', 'output/factCovid.csv', 'output/dimRegion.csv']
dim_dataframe_dict = [dim_date,dim_hospital,factCovid,dimRegion]

# Upload each DataFrame to S3 in a loop
for path, dataframe in zip(file_paths, dim_dataframe_dict):
    upload_dataframe_to_s3(dataframe, path)


File output/dim_date.csv already exists. Deleting it...
File output/dim_hospital.csv already exists. Deleting it...
File output/factCovid.csv already exists. Deleting it...
File output/dimRegion.csv already exists. Deleting it...


In [57]:
#get schema for the dim and fact tables
dim_date.head()

,fips,date,year,month,dayOfWeek
0,49.0,2021-02-20,2021,2,5
1,51.0,2021-02-20,2021,2,5
2,78.0,2021-02-20,2021,2,5
3,50.0,2021-02-20,2021,2,5
4,53.0,2021-02-20,2021,2,5


In [61]:
import pandas as pd

# List of DataFrames
dataframes = [dim_date, dim_hospital, factCovid, dimRegion]
table_names = ['dim_date', 'dim_hospital', 'factCovid', 'dimRegion']

# Loop through the DataFrames
for df, table_name in zip(dataframes, table_names):
    # Get the SQL schema of the subset DataFrame
    schema = pd.io.sql.get_schema(df, table_name)

    # Print the SQL schema
    print(schema)
    print('\n')


CREATE TABLE "dim_date" (
"fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "dayOfWeek" INTEGER
)


CREATE TABLE "dim_hospital" (
"fips" INTEGER,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)


CREATE TABLE "factCovid" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


CREATE TABLE "dimRegion" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)




In [63]:
#pip install redshift_connector

In [64]:
import redshift_connector

In [66]:
import redshift_connector

conn = redshift_connector.connect(
    host='redshift-cluster-1.cdoeoearcpxt.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='dev',
    user='awsuser',
    password='Meenakshi1',
    timeout=120
)



In [67]:
# auto commit to refelect the cahnges to the redshift

In [68]:
conn.autocommit = True

In [19]:
#cursor used for exceution of queries
cursor = redshift_connector.Cursor = conn.cursor()

In [70]:
cursor.execute("""
CREATE TABLE "dim_date" (
"fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "dayOfWeek" INTEGER
)
""")

In [71]:
cursor.execute("""
CREATE TABLE "dim_hospital" (
"fips" INTEGER,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")

In [72]:
cursor.execute("""
CREATE TABLE "factCovid" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
""")

In [77]:
cursor.execute("""
copy dim_date from 's3://meenakshi-test-bucket/output/dim_date.csv'
CREDENTIALS 'aws_iam_role=arn:aws:iam::591034249457:role/service-role/AmazonRedshift-CommandsAccessRole-20230525T105017'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

In [ ]:
cursor.execute("""
copy dimRegion from 's3://meenakshi-test-bucket/output/dimRegion.csv'
CREDENTIALS 'aws_iam_role=arn:aws:iam::591034249457:role/service-role/AmazonRedshift-CommandsAccessRole-20230525T105017'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

cursor.execute("""
copy dim_hospital from 's3://meenakshi-test-bucket/output/dim_hospital.csv'
CREDENTIALS 'aws_iam_role=arn:aws:iam::591034249457:role/service-role/AmazonRedshift-CommandsAccessRole-20230525T105017'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

cursor.execute("""
copy factCovid from 's3://meenakshi-test-bucket/output/factCovid.csv'
CREDENTIALS 'aws_iam_role=arn:aws:iam::591034249457:role/service-role/AmazonRedshift-CommandsAccessRole-20230525T105017'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

In [78]:
cursor.execute("SELECT * FROM stl_load_errors ORDER BY starttime DESC LIMIT 1")
error_details = cursor.fetchall()
print(error_details)

([100, 3, 105932, datetime.datetime(2023, 5, 25, 17, 58, 33, 598886), 1073840563, 10353, 's3://meenakshi-test-bucket/output/dimRegion.csv                                                                                                                                                                                                                 ', 2, '                                                                                                                               ', '          ', '          ', 18, ',Anhui,China,1.0,,,,20210119,289939,,1066.0,,                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [79]:
dimRegion.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York
